# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [24]:
def expected_return(P, rf, mkt_rp):
    x = []
    length = len(P["Beta"])
    beta = 0
    summation = sum(P["Amount Invested"])
    for i in range(length):
        x.append((P["Amount Invested"][i])/summation)
    
    for i in range(length):
        beta += x[i]*P["Beta"][i]
    
    exp_return = rf + beta*mkt_rp
    return exp_return


portfolio = {"Beta": [1.2, 1.4], 
             "Volatility": [0.25, 0.4],
             "Amount Invested": [4000, 6000] }
print(expected_return(portfolio,0.02,0.05))

0.086


Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [37]:
def volatility(P, cor):
    x = []
    length = len(P["Beta"])
    beta = 0
    var = 0
    y = 1
    
    summation = sum(P["Amount Invested"])
    for i in range(length):
        x.append((P["Amount Invested"][i])/summation)
    
    for i in range(length):
        beta += x[i]*P["Beta"][i]
    
    for i in range(length):
        var += ((x[i])**2)*((P["Volatility"][i])**2)
        y *= x[i]*P["Volatility"][i]
    
    variance = var + length*y*cor
    vol = variance**(1/2)
    
    return variance

portfolio = {"Beta": [1.2, 1.4], 
             "Volatility": [0.25, 0.4],
             "Amount Invested": [4000, 6000] }
print(volatility(portfolio,-0.2))

0.058000000000000024


Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [42]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    x = vol/mkt_vol
    exp_return2 = rf + x*mkt_rp
    return exp_return2
print(expected_return2(0.02, 0.05, 0.16, 0.2408))

0.09525


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [52]:
import numpy as np
import pandas as pd

def CAPM_beta(return_path, factors_path):
    return_data = pd.read_csv(return_path)
    return_data["RET"] = pd.to_numeric(return_data["RET"], errors='coerce')

    factors_data = pd.read_csv(factors_path, delim_whitespace=True, skiprows=3)
    factors_data["Mkt-RF"] = factors_data["Mkt-RF"] / 100  # Convert percentage to decimal
    factors_data["RF"] = factors_data["RF"] / 100  # Convert percentage to decimal

    merged_data = pd.merge(return_data, factors_data, how='inner', left_on=["year", "month"], right_on=["date", "date"])

    merged_data["Excess_Return"] = merged_data["RET"] - merged_data["RF"]

    beta_data = merged_data.groupby("CUSIP").apply(lambda x: np.cov(x["Excess_Return"], x["Mkt-RF"])[0, 1] / np.var(x["Mkt-RF"])).reset_index()
    beta_data.columns = ["CUSIP", "BETA"]

    exp_return_data = merged_data.groupby("CUSIP")["Excess_Return"].mean().reset_index()
    exp_return_data.columns = ["CUSIP", "EXP_RETURN"]

    output = pd.merge(beta_data, exp_return_data, how='inner', on="CUSIP")
    return output

#    output = pd.DataFrame(data = {
#        "CUSIP" : [] ,
#        "EXP_RETURN" : [],
#        "BETA" : [],
#    })

#print(CAPM_beta("/Users/hannahong/Downloads/monthlycrsp_mod7.csv", "/Users/hannahong/Downloads/F-F_Research_Data_Factors.CSV"))

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [54]:
def slope(data):
    coefficients = np.polyfit(data["BETA"], data["EXP_RETURN"], 1)
    slope = coefficients[0]
    return slope